<a href="https://colab.research.google.com/github/Thuan-ML/ML-II---Identifying-risky-borrowers-/blob/main/Notebook/Identifying_risky_borrowers_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project: Identifying Risky Borrowers Using Logistic Regression, AdaBoost, LightGBM and XGBoost**

# **1. Introduction**

In this project, we want to identify borrowers who are likely to default on a loan. Using a loan dataset from Kaggle, we frame the problem as a binary classification task, where each borrower is classified as either defaulting or non-defaulting.

We start with a simple Logistic Regression model as a baseline, then apply AdaBoost as an intermediate model. Next, we introduce LightGBM as a faster and more scalable gradient boosting method, and finally use XGBoost as a more advanced method to examine whether increasing model complexity improves the identification of risky borrowers.

# **2. Import Libraries**

In [ ]:
!git clone https://github.com/Thuan-ML/ML-II---Identifying-risky-borrowers-.git

import os
os.chdir("ML-II---Identifying-risky-borrowers-")

Cloning into 'ML-II---Identifying-risky-borrowers-'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 51 (delta 15), reused 11 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 5.35 MiB | 11.85 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [ ]:
!pip install xgboost
!pip install lightgbm
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


# **3. Data Loading**

In [ ]:
df = pd.read_excel("Data/Loan.xlsx")
df.head()

,ApplicationDate,Age,AnnualIncome,CreditScore,EmploymentStatus,EducationLevel,Experience,LoanAmount,LoanDuration,MaritalStatus,...,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,RiskScore
0,2018-01-01,45,39948,617,Employed,Master,22,13152,48,Married,...,3329.000000,0.724972,11,126928,0.199652,0.227590,419.805992,0.181077,0,49.0
1,2018-01-02,38,39709,628,Employed,Associate,15,26045,48,Single,...,3309.083333,0.935132,3,43609,0.207045,0.201077,794.054238,0.389852,0,52.0
2,2018-01-03,47,40724,570,Employed,Bachelor,26,17627,36,Married,...,3393.666667,0.872241,6,5205,0.217627,0.212548,666.406688,0.462157,0,52.0
3,2018-01-04,58,69084,545,Employed,High School,34,37898,96,Single,...,5757.000000,0.896155,5,99452,0.300398,0.300911,1047.506980,0.313098,0,54.0
4,2018-01-05,37,103264,594,Employed,Associate,17,9184,36,Married,...,8605.333333,0.941369,5,227019,0.197184,0.175990,330.179140,0.070210,1,36.0


# **4. Data Overview**

In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ApplicationDate             20000 non-null  datetime64[ns]
 1   Age                         20000 non-null  int64         
 2   AnnualIncome                20000 non-null  int64         
 3   CreditScore                 20000 non-null  int64         
 4   EmploymentStatus            20000 non-null  object        
 5   EducationLevel              20000 non-null  object        
 6   Experience                  20000 non-null  int64         
 7   LoanAmount                  20000 non-null  int64         
 8   LoanDuration                20000 non-null  int64         
 9   MaritalStatus               20000 non-null  object        
 10  NumberOfDependents          20000 non-null  int64         
 11  HomeOwnershipStatus         20000 non-null  object    

,ApplicationDate,Age,AnnualIncome,CreditScore,Experience,LoanAmount,LoanDuration,NumberOfDependents,MonthlyDebtPayments,CreditCardUtilizationRate,...,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,RiskScore
count,20000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,2.000000e+04,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,2045-05-18 12:00:00,39.752600,59161.473550,571.612400,17.522750,24882.867800,54.057000,1.517300,454.292700,0.286381,...,4891.715521,0.799918,5.002650,7.229432e+04,0.239124,0.239110,911.607052,0.402182,0.239000,50.766780
min,2018-01-01 00:00:00,18.000000,15000.000000,343.000000,0.000000,3674.000000,12.000000,0.000000,50.000000,0.000974,...,1250.000000,0.259203,0.000000,1.000000e+03,0.130101,0.113310,97.030193,0.016043,0.000000,28.800000
25%,2031-09-09 18:00:00,32.000000,31679.000000,540.000000,9.000000,15575.000000,36.000000,0.000000,286.000000,0.160794,...,2629.583333,0.727379,3.000000,8.734750e+03,0.213889,0.209142,493.763700,0.179693,0.000000,46.000000
50%,2045-05-18 12:00:00,40.000000,48566.000000,578.000000,17.000000,21914.500000,48.000000,1.000000,402.000000,0.266673,...,4034.750000,0.820962,5.000000,3.285550e+04,0.236157,0.235390,728.511452,0.302711,0.000000,52.000000
75%,2059-01-25 06:00:00,48.000000,74391.000000,609.000000,25.000000,30835.000000,72.000000,2.000000,564.000000,0.390634,...,6163.000000,0.892333,6.000000,8.882550e+04,0.261533,0.265532,1112.770759,0.509214,0.000000,56.000000
max,2072-10-03 00:00:00,80.000000,485341.000000,712.000000,61.000000,184732.000000,120.000000,5.000000,2919.000000,0.917380,...,25000.000000,0.999433,16.000000,2.603208e+06,0.405029,0.446787,10892.629520,4.647657,1.000000,84.000000
std,NaN,11.622713,40350.845168,50.997358,11.316836,13427.421217,24.664857,1.386325,240.507609,0.159793,...,3296.771598,0.120665,2.236804,1.179200e+05,0.035509,0.042205,674.583473,0.338924,0.426483,7.778262


# **5. Split Data into Features and Target Variables**

The task is formulated as a supervised binary classification problem.
Given borrower features X, the objective is to predict the binary outcome y, indicating whether a loan is approved.

In [ ]:
target = "LoanApproved"
# Feature variables
X = df.drop(columns=[target])
# Target variable
y = df[target]


#**6. Train–test split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)



# **7. Data Preprocessing**

##**7.1 Encoding categorical variables**

In [ ]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

X_train, X_test = X_train.align(X_test, axis=1, fill_value=0)

## **7.2 Cleaning datatypes**

In [ ]:
# Drop datetime columns
datetime_cols = X_train.select_dtypes(include=["datetime64[ns]"]).columns
X_train = X_train.drop(columns=datetime_cols)
X_test = X_test.drop(columns=datetime_cols)

# Convert boolean columns to numeric
X_train = X_train.astype(int)
X_test = X_test.astype(int)


##**7.3 Feature scaling**

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



**7.4. Feature selection**

In [ ]:
selector = SelectKBest(score_func=f_classif, k=10)

X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 8 11 21 24 25] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
selected_features = X_train.columns[selector.get_support()]
selected_features

Index(['Age', 'AnnualIncome', 'CreditScore', 'LoanAmount', 'TotalAssets',
       'MonthlyIncome', 'NetWorth', 'MonthlyLoanPayment', 'RiskScore',
       'EducationLevel_High School'],
      dtype='object')

#**8. Baseline model – Logistic Regression**

##**8.1 Train model**

In [ ]:
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train_selected, y_train)

LogisticRegression(max_iter=1000)

##**8.2 Prediction**

In [ ]:
y_pred_lr = model_lr.predict(X_test_selected)

##**8.3 Evaluation**

In [ ]:
acc_lr = accuracy_score(y_test, y_pred_lr)
prec_lr = precision_score(y_test, y_pred_lr)
rec_lr = recall_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr)

print("Logistic Regression")
print("Accuracy:", acc_lr)
print("Precision:", prec_lr)
print("Recall:", rec_lr)
print("F1-score:", f1_lr)

Logistic Regression
Accuracy: 0.9835
Precision: 0.9713983050847458
Recall: 0.9592050209205021
F1-score: 0.9652631578947368


#**9. AdaBoost classifier**

##**9.1 Train AdaBoost with CV + tuning**

In [ ]:
param_grid = {
    "n_estimators": [50, 100, 200],
    "learning_rate": [0.05, 0.1, 0.5]
}

grid_ada = GridSearchCV(
    AdaBoostClassifier(random_state=42),
    param_grid,
    scoring="f1",
    cv=5
)

grid_ada.fit(X_train_selected, y_train)

model_ada = grid_ada.best_estimator_

##**9.2 Prediction**

In [ ]:
y_pred_ada = model_ada.predict(X_test_selected)

##**9.3 Evaluation**

In [ ]:
acc_ada = accuracy_score(y_test, y_pred_ada)
prec_ada = precision_score(y_test, y_pred_ada)
rec_ada = recall_score(y_test, y_pred_ada)
f1_ada = f1_score(y_test, y_pred_ada)

print("AdaBoost")
print("Accuracy:", acc_ada)
print("Precision:", prec_ada)
print("Recall:", rec_ada)
print("F1-score:", f1_ada)

AdaBoost
Accuracy: 0.98525
Precision: 0.9766206163655685
Recall: 0.9612970711297071
F1-score: 0.9688982604111755


##**9.4 Feature Importance**

In [ ]:
feature_importance_ada = pd.Series(
    model_ada.feature_importances_,
    index=selected_features
).sort_values(ascending=False)

feature_importance_ada


,0
RiskScore,0.461376
MonthlyIncome,0.223314
LoanAmount,0.160113
CreditScore,0.068904
TotalAssets,0.044779
NetWorth,0.019127
EducationLevel_High School,0.016276
Age,0.006111
AnnualIncome,0.000000
MonthlyLoanPayment,0.000000


##**10. Gradient Boosting model – XGBoost**

##**10.1 Train XGBoost (with Cross-Validation + Hyperparameter tuning)**

In [ ]:
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [3, 4, 5],
    "learning_rate": [0.05, 0.1]
}

grid_xgb = GridSearchCV(
    XGBClassifier(
        random_state=42,
        eval_metric="logloss",
        use_label_encoder=False
    ),
    param_grid,
    scoring="f1",
    cv=5
)

grid_xgb.fit(X_train_selected, y_train)

model_xgb = grid_xgb.best_estimator_



/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:48:57] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:48:57] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:48:57] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:48:57] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:48:57] WARNING: /w

##**10.2 Prediction**

In [ ]:
y_pred_xgb = model_xgb.predict(X_test_selected)

##**10.3 Evaluation**

In [ ]:
acc_xgb = accuracy_score(y_test, y_pred_xgb)
prec_xgb = precision_score(y_test, y_pred_xgb)
rec_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)

print("XGBoost")
print("Accuracy:", acc_xgb)
print("Precision:", prec_xgb)
print("Recall:", rec_xgb)
print("F1-score:", f1_xgb)

XGBoost
Accuracy: 0.98775
Precision: 0.9798941798941799
Recall: 0.9686192468619247
F1-score: 0.9742240925828511


##**10.4 Feature importance**

In [ ]:
feature_importance_xgb = pd.Series(
    model_xgb.feature_importances_,
    index=selected_features
).sort_values(ascending=False)

feature_importance_xgb


,0
RiskScore,0.748543
MonthlyIncome,0.059143
NetWorth,0.045372
LoanAmount,0.038810
CreditScore,0.034271
EducationLevel_High School,0.024315
TotalAssets,0.014713
AnnualIncome,0.014349
Age,0.011471
MonthlyLoanPayment,0.009013


#**11. LightGBM classifier**

##**11.1 Train model**

In [ ]:
model_lgb = LGBMClassifier(random_state=42)
model_lgb.fit(X_train_selected, y_train)

[LightGBM] [Info] Number of positive: 3824, number of negative: 12176
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002103 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1873
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.239000 -> initscore=-1.158170
[LightGBM] [Info] Start training from score -1.158170


LGBMClassifier(random_state=42)

##**11.2 Prediction**

In [ ]:
y_pred_lgb = model_lgb.predict(X_test_selected)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


##**11.3 Evaluation**

In [ ]:
acc_lgb = accuracy_score(y_test, y_pred_lgb)
prec_lgb = precision_score(y_test, y_pred_lgb)
rec_lgb = recall_score(y_test, y_pred_lgb)
f1_lgb = f1_score(y_test, y_pred_lgb)

print("LightGBM")
print("Accuracy:", acc_lgb)
print("Precision:", prec_lgb)
print("Recall:", rec_lgb)
print("F1-score:", f1_lgb)


LightGBM
Accuracy: 0.988
Precision: 0.9809322033898306
Recall: 0.9686192468619247
F1-score: 0.9747368421052631


##**11.4 Feature Importance**

In [ ]:
feature_importance_lgb = pd.Series(
    model_lgb.feature_importances_,
    index=selected_features
).sort_values(ascending=False)

feature_importance_lgb / feature_importance_lgb.sum()

,0
RiskScore,0.154667
LoanAmount,0.154667
MonthlyIncome,0.137333
CreditScore,0.130333
NetWorth,0.114667
TotalAssets,0.085000
MonthlyLoanPayment,0.074667
Age,0.064667
AnnualIncome,0.054000
EducationLevel_High School,0.030000


#**12. Model comparison**

In [ ]:
results = pd.DataFrame({
    "Model": ["Logistic Regression", "AdaBoost", "LightGBM", "XGBoost"],
    "Accuracy": [acc_lr, acc_ada, acc_lgb, acc_xgb],
    "Precision": [prec_lr, prec_ada, prec_lgb, prec_xgb],
    "Recall": [rec_lr, rec_ada, rec_lgb, rec_xgb],
    "F1-score": [f1_lr, f1_ada, f1_lgb, f1_xgb]
})

results



,Model,Accuracy,Precision,Recall,F1-score
0,Logistic Regression,0.98350,0.971398,0.959205,0.965263
1,AdaBoost,0.98525,0.976621,0.961297,0.968898
2,LightGBM,0.98800,0.980932,0.968619,0.974737
3,XGBoost,0.98775,0.979894,0.968619,0.974224


We tested four models to classify whether a borrower is risky or not: Logistic Regression, AdaBoost, LightGBM, and XGBoost.

* Logistic Regression serves as a strong baseline, with an accuracy of 98.35% and precision of 97.13%. It performs well and is easy to interpret.

* AdaBoost improves slightly over the baseline, achieving 98.52% accuracy and a higher recall (96.13%), meaning it identifies more risky borrowers correctly.

* LightGBM, a faster and more scalable boosting method, reaches 98.80% accuracy, and F1-score of 0.9747, the highest among all.

* XGBoost also performs competitively, with 98.77% accuracy and similar recall and F1-score to LightGBM.

 to sum up: Logistic Regression works well and gives a strong baseline. AdaBoost performs similarly, with only small improvements. LightGBM and XGBoost achieve the best results overall. This demonstrates how increasing model complexity from linear to boosted trees can lead to better performance on structured data.